In [0]:
import datetime

2023-04-06 06:56:26,131 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
%run ./common_utils

2023-04-06 06:56:26,240 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:26,340 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
%run ./job_level_scripting

2023-04-06 06:56:26,440 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:26,742 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:26,541 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
%run ./wf_level_scripting

2023-04-06 06:56:26,842 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:26,641 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:26,943 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,044 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,445 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,143 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,244 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,546 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,346 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


2023-04-06 06:56:27,664 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def exec_job_prescript(config_data,table):
    job_num = table.get('job_num')
    current_batch_start_date = datetime.datetime.now()
    batch_sub_batch_id = job_pre_script(config_data,job_num,current_batch_start_date)
    return batch_sub_batch_id

2023-04-06 06:56:27,751 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def exec_job_postscript(config_data,table,batch_sub_batch_id):
    # status_num = 3 indicates the job completed successfully
    status_num = 3
    job_num = table.get('job_num')
    current_batch_end_date = datetime.datetime.now()
    job_post_script(config_data,job_num,batch_sub_batch_id,status_num,current_batch_end_date)
    return 1

2023-04-06 06:56:27,848 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def get_buss_start_dt(config_data,table):
    wf_name = table.get('wf_name')
    buss_start_dt_query = f"select case when max(business_end_date) is null then '1800-01-01' else max(business_end_date) end max_business_end_date from {config_data.get('source').get('audit_schema_nm')}.audit_log where table_name = '{wf_name}' "
    log.debug(f"buss_start_dt_query: {buss_start_dt_query}")
    buss_start_dt = spark.sql(buss_start_dt_query).collect()[0][0]
    return buss_start_dt

2023-04-06 06:56:27,948 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def get_src_count(query):
    return spark.sql(query).collect()[0][0]

2023-04-06 06:56:28,048 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def exec_wf_postscript(config_data,table,batch_start_date,batch_end_date,src_count,buss_start_dt,batch_sub_batch_id):
    # status_num = 3 indicates the job completed successfully
    status_num = 3
    wf_name = table.get('wf_name')
    job_num = table.get('job_num')
    elapsed_time = (batch_end_date - batch_start_date).total_seconds()
    log.info(f"{wf_name} executed in {round(elapsed_time,2)} sec | {round(int(elapsed_time) / 60,2)}  min")
    trgt_cnt_query = f"select count(1) from {config_data.get('source').get('catalog_nm')}.{wf_name}"
    trgt_cnt_query = f"{trgt_cnt_query} where {table.get('incr_indentifier_cols')} > '{buss_start_dt}' "
    log.debug(f"trgt_cnt_query: {trgt_cnt_query}")
    target_count = spark.sql(trgt_cnt_query).collect()[0][0];
    log.info(f"target_count: {target_count}")
    init_ssi_audit(config_data,table,job_num,batch_start_date,batch_end_date,src_count,target_count,buss_start_dt,batch_sub_batch_id,status_num)
    return 1

2023-04-06 06:56:28,150 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def exec_wf_postscript_exception(config_data,table,batch_start_date,batch_end_date,src_count,buss_start_dt,batch_sub_batch_id,failure_point="null",technical_error_desc="null"):
    # status_num = 2 indicates the job has errors
    status_num = 2
    wf_name = table.get('wf_name')
    job_num = table.get('job_num')
    elapsed_time = (batch_end_date - batch_start_date).total_seconds()
    log.info(f"{wf_name} executed in {round(elapsed_time,2)} sec | {round(int(elapsed_time) / 60,2)}  min")
#     trgt_cnt_query = f"select count(1) from {config_data.get('source').get('catalog_nm')}.{wf_name}"
#     trgt_cnt_query = f"{trgt_cnt_query} where {table.get('incr_indentifier_cols')} > '{buss_start_dt}' "
#     log.debug(f"trgt_cnt_query: {trgt_cnt_query}")
#     target_count = spark.sql(trgt_cnt_query).collect()[0][0];
    target_count = "null"
    log.info(f"target_count: {target_count}")
    init_ssi_audit(config_data,table,job_num,batch_start_date,batch_end_date,src_count,target_count,buss_start_dt,batch_sub_batch_id,status_num,failure_point=failure_point,technical_error_desc=technical_error_desc)
    return 1

2023-04-06 06:56:28,250 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 
